In [1]:
# ! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [3]:
from fastai.vision.all import *
from fastbook import *

In [10]:
#time = torch.arange(0,20).float();
#speed = torch.randn(20)*3 + 0.75*(time-9.5)**2 + 1
#plt.scatter(time,speed);

In [11]:
#time, speed

In [52]:
time = torch.arange(0,20).float();
#speed = torch.randn(20)*3 + 0.75*(time-9.5)**2 + 1
speed = tensor([72.5971, 56.6511, 46.5895, 31.6208, 20.7394, 17.0928, 10.7236,  5.2996, -0.0999,  2.0161, -0.4291,  4.0752,  7.5875, 11.4025, 17.2123, 23.0234, 36.4538, 41.0426, 57.7493, 70.2265])
params = tensor([-1.3392, -1.3702, -0.3218], requires_grad=True)

def f(t, params):
    a,b,c = params
    return a*(t**2) + (b*t) + c

def mse(preds, targets): return ((preds-targets)**2).mean()


# trying to calculate gradiant
preds = f(time, params)
loss = mse(preds, speed) # 70006.9766
# 2*(preds-speed).mean() # -410.6172
# loss.backward()
params.grad
#   tensor([-88315.0469,  -5684.3477,   -410.6172]) # after one run of loss.backward()

tensor([-88315.0469,  -5684.3477,   -410.6172])

In [96]:
# Derivative of f(t, params) w.r.t.
#   a: t^2
#   b: t
#   c: 1

# Derivative of Loss w.r.t. preds
# we can either choose to use mean() or divide by n, len(time) aka preds.shape[0]
#   loss' = mse'(preds, speed) = 2 * (preds-speed).mean() # tensor scalar
#   or
#   loss' = (2 / len(time)) * (preds - speed) # tensor vector

loss_p = (2 / len(time)) * (preds - speed)
grad_a = (loss_p*(time**2)).sum()
grad_b = (loss_p*time).sum()
grad_c = loss_p.sum()
grad_a, grad_b, grad_c, loss_p

(tensor(-88315.0469, grad_fn=<SumBackward0>),
 tensor(-5684.3477, grad_fn=<SumBackward0>),
 tensor(-410.6172, grad_fn=<SumBackward0>),
 tensor([ -7.2919,  -5.9682,  -5.5009,  -4.8106,  -4.7969,  -5.7746,  -6.7478,  -8.0834,  -9.6892, -12.3145, -14.7515, -18.1512, -21.7196, -25.5862, -29.9200, -34.5218, -40.1534, -45.1687, -51.6636,
         -58.0033], grad_fn=<MulBackward0>))

Taking the gradient of the loss wrt parameter 'b':
    grad_b = -56684.3477
- a negative gradient suggests that increasing the value of 'b' would decrease the loss
    - and we're trying to _minimize_ loss
- the gradient tells you how much the loss will change for an infinitesimally small change in the parameter
    - it's a local linear approximation; it's accurate for very small changes around the curren tvalue of the parameter
- non-linear effects
    - for larger changes in the parameter the relationship between the parameter and the loss may not be linear
- the learning rate is crucial in determining how much 'b' actually changes in each step

In [32]:
"""
loss.backward()
how can we calculate this?

f(t, params) -> f(t, a, b,c )
f(t, a, b, c) = at^2 + bt + c
f = at^2 + bt + c

f'/da = t^2
f'/db = t
f'/dc = 1

loss = mse(preds, speed) = ((preds-speed)**2).mean()
loss' = 2*(preds-speed).mean()

grad_a (gradient for 'a')
grad_a -> f'/da * loss' # things get weird with matrices and such (e,g. placement/order of mean() and sum())
    (2/n)(preds-speed).sum()*times^2
grad_a = ((2 / len(time)) * (preds - speed)*(time**2)).sum()

"""

'\nloss.backward()\nhow can we calculate this?\n\nf(t, params) -> f(t, a, b,c )\nf(t, a, b, c) = at^2 + bt + c\n\n\n\n\n\n'

In [22]:
params, time, speed, preds, loss

(tensor([-1.3392, -1.3702, -0.3218], requires_grad=True),
 tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19.]),
 tensor([67.7380, 56.9534, 40.5161, 33.9169, 29.4810, 19.2231,  5.8783,  2.2979,  2.2794,  6.0937,  3.1517,  4.4155,  9.1120, 10.2432, 10.7701, 26.4638, 31.5615, 46.2868, 53.1275, 70.5979]),
 tensor([-3.2180e-01, -3.0312e+00, -8.4190e+00, -1.6485e+01, -2.7230e+01, -4.0653e+01, -5.6754e+01, -7.5534e+01, -9.6992e+01, -1.2113e+02, -1.4794e+02, -1.7744e+02, -2.0961e+02, -2.4446e+02,
         -2.8199e+02, -3.2219e+02, -3.6508e+02, -4.1064e+02, -4.5889e+02, -5.0981e+02], grad_fn=<AddBackward0>),
 tensor(69850.4141, grad_fn=<MeanBackward0>))